# Bushfire Attack Level Toolbox - fixing dependencies

An undocumented dependency exists in the Bushfire Attack Level Toolbox (BAL Toolbox) code [http://github.com/GeoscienceAustralia/BAL], where the `scipy.ndimage` package is required to calculate the gradient of the input digital elevation model. 

However, because the BAL Toolbox is built on the Arcpy module, it is difficult to install the Scipy library. The `arcpy` module is built against a specific (and modified) version of `numpy`. Finding a version of `scipy` that matches the `numpy` version is nigh on impossible. Upgrading the installed version of `numpy` is also not possible, as this breaks the `arcpy` installation.

This notebook is to test the option of replacing the function from `scipy.ndimage` with the `numpy.gradient` function. 

In [1]:
%matplotlib inline

import numpy as np
import scipy.ndimage as scnd

import matplotlib.pyplot as plt


## Some definitions

The BAL Toolbox uses the `scipy.ndimage.sobel` function to calculate the gradient of the input DEM in the x- and y-directions. The Sobel filter is in fact an operator suited to edge detection, which calculates an approximation to the gradient.  The Sobel filter is defined as:

$\mathbf{G}_x = \begin{bmatrix} 
 +1 & 0 & -1  \\
+2 & 0 & -2 \\
+1 & 0 & -1 
\end{bmatrix} * \mathbf{A}
\quad
\mbox{and}
\quad   
\mathbf{G}_y = \begin{bmatrix} 
 +1 & +2 & +1\\
 0 & 0 & 0 \\
-1 & -2 & -1
\end{bmatrix} * \mathbf{A}$

where  $*$ here denotes the 2-dimensional signal processing convolution operation.

Since the Sobel kernels can be decomposed as the products of an averaging and a differentiation kernel, they compute the gradient with smoothing. For example, $\mathbf {G_{x}}$  can be written as:

$\begin{bmatrix}+1&0&-1\\+2&0&-2\\+1&0&-1\end{bmatrix}={\begin{bmatrix}1\\2\\1\end{bmatrix}}{\begin{bmatrix}+1&0&-1\end{bmatrix}}$

In contrast, the `numpy.gradient` operator calculates a second order accurate central difference in the interior and either first difference or second order accurate one-sides (forward or backwards) difference at the boundaries. This has no smoothing applied.

## Test set up

We set up a simple grid, representing an elevation model, with a uniform north-south ridge in the middle of the domain. Some random variation is added, then a smoothing filter passed over the grid. 

We then calculate the slope using `numpy.gradient` and `scipy.ndimage.sobel` and compare the results.

In [34]:
np.random.seed(1)


nx = 401
ny = 401
dx = 25.
dy = 25.
ht = np.zeros((nx, ny))

hill_width=2000 # Hill width (m) - same as hill_width in WRF emles code!
idx = hill_width/dx
xs = nx/2 - idx
xe = xs + 2.*idx
ys = ny/2 - idx
ye = ys + 2.*idx
xx, yy = np.meshgrid(np.arange(ny), np.arange(nx))
xm = xx * dx
ym = yy * dx

for i in range(nx):
    for j in range(int(ys), int(ye)):
        ht[i,j] =  250. * 0.5 * (1. + np.cos(2*np.pi/(ye-ys) *(j-ys)+ np.pi))
        
r = np.random.normal(0, 0.5,size=(nx, ny))

ht += r
f = np.ones((3,3))/9.
ht = scnd.convolve(ht, f)

plt.contourf(xm, ym, ht, cmap='terrain')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.title("Elevation (m)")

First, calculate the slope using `numpy.gradient`. In this case, we can automatically include the grid spacing in the function call. 

In [35]:
slope_x, slope_y = np.gradient(ht, dx)
slope_np = np.hypot(slope_x, slope_y)
print(slope_y.max())
print(slope_y.min())


# In[11]:

plt.contourf(xm, ym, slope_np, cmap='seismic')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.title("Hill slope (fractional)")

Now calculate the slope using the Sobel filter. Here, we need to account for the fact that the convolution filter applied needs to be normalised, and the grid spacing also needs to be included explicitly.

In [36]:
dzdx = scnd.sobel(ht, axis=1)/(8.*dx)
dzdy = scnd.sobel(ht, axis=0)/(8.*dy)
slope_sc = np.hypot(dzdx, dzdy)

plt.contourf(xm, ym, slope_sc, cmap='seismic')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.title("Hill slope (fractional)")

Finally, we take the difference of the two.

In [37]:
plt.contourf(xm, ym, slope_sc - slope_np, cmap='seismic')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.title("Hill slope difference")

In [38]:
plt.contourf(xm, ym, (slope_sc - slope_np)/slope_sc, cmap='seismic')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.title("Hill slope difference (fractional)")

In the case of no random variation to the input elevation grid, the results are indistiguishable.

In [39]:
import numexpr
RADIANS_PER_DEGREE = np.pi/180.
aspect_array = numexpr.evaluate(
        "(450 - arctan2(dzdy, -dzdx) / RADIANS_PER_DEGREE) % 360")

aspect_np = np.mod((450. - np.arctan2(dzdy, -dzdx)/RADIANS_PER_DEGREE), 360.)



In [40]:
plt.contourf(xm, ym, (aspect_array - aspect_np), cmap='seismic')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.title("Hill aspect difference")